In [ ]:
#| default_exp training_module

In [ ]:
#| export
from relax.import_essentials import *
from relax.methods.counternet import CounterNetTrainingModule, CounterNetTrainingModuleConfigs, partition_trainable_params
from relax.utils import grad_update
from copy import deepcopy
from functools import partial

In [ ]:
#| export
def l_inf_proj(x: jnp.ndarray, eps: float, cat_idx: Optional[int] = None):
    if cat_idx is None:
        return x.clip(-eps, eps)
    else:
        return jnp.concatenate([
            x[:, :cat_idx].clip(-eps, eps), # clip continuous features only
            x[:, cat_idx:]
        ], axis=-1)

In [ ]:
#| exporti
def filter_params(params):
    return hk.data_structures.filter(
        lambda m, n, v: m == 'counter_net_model/Predictor/dense_block/linear' and n == 'w', params
    )

In [ ]:
class Attack(ABC):
    def __init__(
        self,
        keys: hk.PRNGSequence,
        pred_loss_fn: Callable[[hk.Params, random.PRNGKey, Tuple[jnp.DeviceArray, jnp.DeviceArray], bool], jnp.DeviceArray],
        adv_loss_fn, # ignored 
        n_steps: int, # attacker steps
        k: int, # inner steps
        epsilon: float,
        adv_lr: float,
        apply_fn: Callable, # apply_fn(x, cf, hard=False)
        cat_idx # ignored
    ):
        self.keys = keys
        self.pred_loss_fn = pred_loss_fn
        self.adv_loss_fn = adv_loss_fn
        self.n_steps = n_steps
        self.k = k
        self.epsilon = epsilon
        self.adv_lr = adv_lr
        self.apply_fn = apply_fn
        self.cat_idx = cat_idx

    def step(
        self,
        params: hk.Params,
        x: jnp.ndarray, 
        y: jnp.ndarray,
    ) -> hk.Params:
        raise NotImplementedError
    
    __call__ = step

In [ ]:
#| export
class RandomAttack(Attack):
    def step(self, params, x, y):
        # init delta randomly
        delta = random.uniform(
            key=next(self.keys), shape=x.shape, 
            minval=-self.epsilon, maxval=self.epsilon)
        # create optimizer
        opt = optax.adam(learning_rate=self.adv_lr)
        opt_state = opt.init(params)

        for _ in range(self.n_steps):
            x = self.apply_fn(x, x + delta, hard=False)
            for _ in range(self.k):
                loss, grads = jax.value_and_grad(self.pred_loss_fn)(
                        params, next(self.keys), (x, y), False)
                params, opt_state = grad_update(grads, params, opt_state, opt)

            delta = random.uniform(
                key=next(self.keys), shape=x.shape, 
                minval=-self.epsilon, maxval=self.epsilon)
            
        return params

In [ ]:
class BilevelAttack(Attack):
    def step(self, params: hk.Params, x: jnp.ndarray, y: jnp.ndarray) -> hk.Params:
        # alpha is the delta's learning rate
        alpha = self.epsilon * 2.5 / self.n_steps
        # init delta randomly
        delta = random.uniform(
            key=next(self.keys), shape=x.shape, 
            minval=-self.epsilon, maxval=self.epsilon)
        # create optimizer
        opt = optax.adam(learning_rate=self.adv_lr)
        opt_state = opt.init(params)

        def attacker_fn(
            delta: jnp.ndarray,
            params: hk.Params,
            opt_state: optax.OptState,
            rng_key: random.PRNGKey,
            batch: Tuple[jnp.ndarray, jnp.ndarray],
            opt: optax.GradientTransformation
        ):
            x, y = batch
            for _ in range(self.k):
                # inner unrolling steps
                _x = self.apply_fn(x, x + delta, hard=False)
                grads = jax.grad(self.pred_loss_fn)(params, rng_key, (_x, y), is_training=True)
                params, opt_state = grad_update(grads, params, opt_state, opt)

            # TODO: x or _x?
            loss = self.adv_loss_fn(params, rng_key, x, is_training=False)
            return loss, (params, opt_state)

        for i in range(self.n_steps):
            g, (params, opt_state) = jax.grad(attacker_fn, has_aux=True)(
                delta, params, opt_state, next(self.keys), (x, y), opt)
            delta = delta + alpha * jnp.sign(g)
            delta = l_inf_proj(delta, self.epsilon, self.cat_idx)
        return params
        

In [ ]:
# TODO: unit tests for attacks 
cfnet = CounterNetTrainingModule(CounterNetTrainingModuleConfigs())
cfnet.init_net_opt(random.PRNGKey(0), cfnet.x_train[:1], cfnet.y_train[:1])

AttributeError: 'CounterNetTrainingModule' object has no attribute 'x_train'

In [ ]:
#| export
def bilevel_adv_step(
    params: hk.Params,
    keys: hk.PRNGSequence,
    batch: Tuple[jnp.DeviceArray, jnp.DeviceArray], # (x, y)
    pred_loss_fn: Callable, 
    # pred_loss_fn(params, rng_key, batch, is_training=True)
    adv_loss_fn:  Callable, 
    # adv_loss_fn(params, rng_key, x, is_training=True)
    n_steps: int, # attacker steps
    k: int, # inner steps
    epsilon: float,
    adv_lr: float,
    apply_fn: Callable, # apply_fn(x, cf, hard=False)
    cat_idx: int
):
    """virtual data shift algorithm"""
    x, y = batch
    # alpha
    alpha = epsilon * 2.5 / n_steps
    # init delta randomly
    delta = random.uniform(key=next(keys), shape=x.shape, minval=-epsilon, maxval=epsilon)
    # create optimizer
    opt = optax.adam(learning_rate=adv_lr)
    opt_state = opt.init(params)
    # val_loss = self.adv_loss_fn(params, rng_key, x, delta=None, is_training=False)

    def attacker_fn(
        delta: jnp.ndarray,
        params: hk.Params,
        opt_state: optax.OptState,
        rng_key: random.PRNGKey,
        batch: Tuple[jnp.ndarray, jnp.ndarray],
        opt: optax.GradientTransformation
    ):
        x, y = batch
        for _ in range(k):
            # inner unrolling steps
            x = apply_fn(x, x + delta, hard=False)
            grads = jax.grad(pred_loss_fn)(params, rng_key, batch, is_training=True)
            params, opt_state = grad_update(grads, params, opt_state, opt)

        loss = adv_loss_fn(params, rng_key, x, is_training=False)
        return loss, (params, opt_state)

    for i in range(n_steps):
        g, (params, opt_state) = jax.grad(attacker_fn, has_aux=True)(
            delta, params, opt_state, next(keys), batch, opt)
        delta = delta + alpha * jnp.sign(g)
        delta = l_inf_proj(delta, epsilon, cat_idx)
    return params

In [ ]:
#| export
class RoCourseNetTrainingConfigs(CounterNetTrainingModuleConfigs):
    epsilon: float = 0.1
    n_steps: int = 7
    k: int = 2
    adv_lr: float
    random_perturbation: bool = False
    seed: int = 42

    @property
    def keys(self):
        return hk.PRNGSequence(self.seed)

In [ ]:
#| export
class RoCourseNetTrainingModule(CounterNetTrainingModule):
    name = "RoCourseNet"

    def __init__(self, m_configs: Dict[str, Any]):
        super().__init__(m_configs)
        self.configs = RoCourseNetTrainingConfigs(**m_configs)
    
    def adv_loss_fn(
        self,
        params: hk.Params, 
        rng_key: random.PRNGKey, 
        x: jnp.DeviceArray, 
        is_training: bool = True
    ):
        y_pred, cf, cf_y = self.forward(params, rng_key, x, is_training)
        y_prime = 1 - jnp.round(y_pred)
        return self.loss_fn_2(cf_y, y_prime)

    def bilevel_adv_step(
        self,
        params: hk.Params,
        rng_key: random.PRNGKey,
        batch: Tuple[jnp.ndarray, jnp.ndarray]
    ) -> hk.Params:
        if self.configs.random_perturbation:
            adv_step_fn = random_adv_step
        else:
            adv_step_fn = bilevel_adv_step

        return adv_step_fn(
            params=params,
            keys=rng_key,
            batch=batch,
            pred_loss_fn=self.pred_loss_fn,
            adv_loss_fn=self.adv_loss_fn,
            n_steps=self.configs.n_steps, 
            k=self.configs.k,
            epsilon=self.configs.epsilon,
            adv_lr=self.configs.adv_lr,
            apply_fn=self._data_module.apply_constraints,
            cat_idx=self._data_module.cat_idx
        )

    def exp_loss_fn(
        self,
        trainable_params: hk.Params,
        non_trainable_params: hk.Params,
        aux_params: hk.Params,
        rng_key: random.PRNGKey,
        batch: Tuple[jnp.DeviceArray, jnp.DeviceArray],
        is_training: bool = True
    ):
        # merge trainable and non_trainable params
        params = hk.data_structures.merge(trainable_params, non_trainable_params)
        x, y = batch
        y_pred, cf = self.net.apply(params, rng_key, x, is_training=is_training)
        cf = self._data_module.apply_constraints(x, cf, hard=not is_training)
        # compute cf_y on shifted model
        cf_y, _ = self.net.apply(aux_params, rng_key, cf, is_training=is_training)
        y_prime = 1 - jnp.round(y_pred)
        loss_2, loss_3 = self.loss_fn_2(cf_y, y_prime), self.loss_fn_3(x, cf)
        return self.configs.lambda_2 * loss_2 + self.configs.lambda_3 * loss_3

    def explainer_step(self, params, aux_params, opt_state, rng_key, batch):
        trainable_params, non_trainable_params = partition_trainable_params(
            params, trainable_name='counter_net_model/Explainer'
        )
        grads = jax.grad(self.exp_loss_fn)(
            trainable_params, non_trainable_params, aux_params, rng_key, batch)
        upt_trainable_params, opt_state = grad_update(
            grads, trainable_params, opt_state, self.opt_2)
        upt_params = hk.data_structures.merge(upt_trainable_params, non_trainable_params)
        return upt_params, opt_state

    @partial(jax.jit, static_argnames=['self'])
    def _training_step(self,
        params: hk.Params,
        opts_state: Tuple[optax.GradientTransformation, optax.GradientTransformation],
        rng_key: random.PRNGKey,
        batch: Tuple[jnp.array, jnp.array]
    ):
        opt_1_state, opt_2_state = opts_state
        params, opt_1_state = self._predictor_step(params, opt_1_state, rng_key, batch)
        aux_params = self.bilevel_adv_step(params, rng_key, batch)
        upt_params, opt_2_state = self.explainer_step(params, aux_params, opt_2_state, rng_key, batch)
        return upt_params, (opt_1_state, opt_2_state)

    def _training_step_logs(self, params, rng_key, batch):
        x, y = batch
        logs = super()._training_step_logs(params, rng_key, batch)
        adv_loss = self.adv_loss_fn(params, rng_key, x, is_training=False)
        logs.update({
            'train/adv_loss': adv_loss
        })
        return logs

In [ ]:
m_configs = {
    "enc_sizes": [50,10],
    "dec_sizes": [10],
    "exp_sizes": [50, 50],
    "dropout_rate": 0.3,
    'lr': 0.003,
    "lambda_1": 1.0,
    "lambda_3": 0.1,
    "lambda_2": 0.2,
}
t_configs = {
    'n_epochs': 100,
    'monitor_metrics': 'val/val_loss'
}
data_configs = {
    "data_dir": "../assets/data/loan/year=2009.csv",
    "data_name": "adult",
    "batch_size": 128,
    'sample_frac': 0.1,
    "continous_cols": [
        "NoEmp", "NewExist", "CreateJob", "RetainedJob", "DisbursementGross", "GrAppv", "SBA_Appv"
    ],
    "discret_cols": [
        "State", "Term", "UrbanRural", "LowDoc", "Sector_Points"
    ],
}


In [ ]:
from cfnet.datasets import TabularDataModule
from cfnet.train import train_model


dm = TabularDataModule(data_configs)
cfnet = RoCourseNetTrainingModule(m_configs)

params, opt_state = train_model(
    cfnet, dm, 
    t_configs
)

Epoch 99: 100%|██████████| 7/7 [00:00<00:00, 92.79batch/s, train/train_loss_1=0.030241895, train/train_loss_2=1.4216381e-07, train/train_loss_3=0.030580275]


In [ ]:
from cfnet.evaluate import evaluate_cfs

cf_res = cfnet.generate_cf_results(dm, params, random.PRNGKey(0))
evaluate_cfs(
    cf_res, return_df=True
)[1]

,acc,validity,proximity
RoCourseNet,0.7820559,1.0,6.7352805
